# Getting Started with Private AI Agents with Cloudera AI
It's 2025 and AI agents are the hot commodities in the AI market. Giving large language models (LLMs) the ability to use tools to complete actions opens up a huge range of use cases. At the same time, it's important to ensure that the AI agents we build are secure. 

In this notebook you will get familiar the CrewAI library and how we can build fully private AI agents with CrewAI and Cloudera's AI Inference Service. Powered by Nvidia NeMo Inference Microservices (NIM), Cloudera's AI Inference Service gives you the ability to deploy private LLMs within your infrastructure. This notebook is designed to walk you through the setup and application of these tools in an agentic worfklow context to create a robust and intelligence system. 

- **Configure the AI Inference Service & CrewAI:** Large Language Models (LLMs) deployed via Cloudera's AI Inference Service have endpoints that are compatible with the OpenAI spec, and can work with a range of popular libraries that are built to work with OpenAI endpoints. In the example below, we deployed the LLama 3.1 8B Instruct LLM, which will serve as the backbone of our AI agents, providing the computational power and linguistic understanding necessary for processing complex queries.
- **Set up CrewAI Agents and Tasks:** We will leverage the CrewAI library to set up our AI agents with clear roles, backstories and goals. We will also configure tasks for each agent to complete and identify tools that they can leverage to complete their tasks.
- **Create Sequential Agentic Workflow:** Next, we will orchestrate these agents and tasks into a crew and workflow for them to complete.
- **Running a Practical Example:** We will then run our example workflow by providing inputs for the agentic workflow to consider and then complete the tasks they are configured to do.

## 1 - Configuring the AI Inference Service & CrewAI

First, let's setup CrewAI with the LLM hosted by the AI Inference Service.

In [10]:
# Import our key libraries and methods
from crewai import Agent, Task, Crew, Process, LLM
import os
import litellm
import json
import warnings
warnings.filterwarnings('ignore')

Below, we obtain the Access Token that we need to use the AI Inference Service endpoint

In [11]:
CAII_API_KEY=json.load(open("/tmp/jwt"))["access_token"]

Next, we set the model endpoint variables.

Note: For the **"CAII_API_BASE_URL"** environment variable, it must not include the **"/chat/completions"** suffix.

For the model name, it is important to add the **"openai/"** prefix, since CrewAI leverages LiteLLM to be able to support various LLMs, and the **"openai/"** prefix tells LiteLLM that we are working with an OpenAI-compatible endpoint.

In [12]:
litellm.set_verbose=False

llm = LLM(
    model="openai/" + os.getenv("CAII_MODEL_NAME"),
    api_key=CAII_API_KEY,
    base_url=os.getenv("CAII_API_BASE_URL")
)

## 2 - Set Up CrewAI Agents and Tasks

Next, let's set up the relevant Agents and Tasks using the CrewAI library. We will define 2 agents - One Researcher and One Reporting Analyst, and they will be tasked to research the topic assigned to them. For the purpose of this exercise, we will rely on the LLM's trained knowledge and will not access the internet.

In [13]:
from textwrap import dedent

# First we define the Researcher Agent
researcher_agent = Agent(
    role=dedent((
        """
        {topic} Senior Data Researcher
        """)), # Think of this as the job title
    backstory=dedent((
        """
        You are a seasoned researcher with a knack for uncovering the latest developments in {topic}.

        You are known for your ability to find the most relevant information and present it in a clear and concise manner.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        Uncover cutting-edge developments in {topic}. Try to keep final answers in markdown format.
        """)), # This is the goal that the agent is trying to achieve
    allow_delegation=False,
    max_iter=2,
    max_retry_limit=3,
    max_retries=3,
    llm=llm,
    verbose=True
)

# Next, we define the Reporting Analyst
reporting_analyst = Agent(
    role=dedent((
        """
        {topic} Reporting Analyst
        """)),
    backstory=dedent((
        """
        You are a meticulous analyst with a keen eye for detail. You are known for your ability to turn complex data into clear and concise reports,
        making it easy for others to understand and act on the information you provide.
        """)),
    goal=dedent((
        """
        Create detailed reports based on {topic} data analysis and research findings.
        """)), # This is the goal that the agent is trying to achieve
    allow_delegation=False,
    max_iter=2,
    max_retry_limit=3,
    max_retries=3,
    llm=llm,
    verbose=True
)

Next, let's define the tasks for our agents to complete.

In [14]:
# First, let's create a task for the researcher agent. Note below that we define which agent completes which task.
research_task = Task(
    description=dedent((
        """
        Conduct thorough research about {topic}

        Make sure you find any interesting and relevant information given the current year is 2024.
        """)),
    expected_output=dedent((
        """
        A list with 2 to 4 bullet points of the most relevant information about {topic}
        """)),
    agent=researcher_agent
)

reporting_task = Task(
    description=dedent((
        """
        Review the context you got and expand each topic into a full section for a report.
        Make sure the report is detailed and contains any and all relevant information.
        """)),
    expected_output=dedent((
        """
        A fully fledge reports with the mains topics, each with a full section of information.
        Formatted as markdown without '```'
        """)),
    agent=reporting_analyst,
    output_file='report.md',
    context=[research_task]
)

## 3 - Create Sequential Agentic AI Workflow

Now, let's put all of this together by placing the discrete agents and tasks into a crew.

In [15]:
crew = Crew(
        agents=[researcher_agent, reporting_analyst],
        tasks=[research_task, reporting_task],
        verbose=True,  # You can set it to True or False
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.sequential
    )

2025-01-21 03:37:46,502 - 139834617546560 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## 4 - Running a Practical Example

Now, let's put all of this together by placing the discrete agents and tasks into a crew. First, we define an input topic. Feel free to change the topic to something else.

In [16]:
# Notice here that the object we pass maps to the {topic} field we added to the agents and tasks.
inputs = {
        "topic": "Best Cities in the US",
}

Next, we create a function to run the crew with the given input. We have enabled verbose logging to help you see the agents execute each task.

In [17]:
def run_crew():
    result = crew.kickoff(inputs=inputs)
run_crew()

# Agent: 
## Task: 
Conduct thorough research about Best Cities in the US

Make sure you find any interesting and relevant information given the current year is 2024.



# Agent: 
## Final Answer: 
* As of 2024, the Best Cities in the US are determined by factors such as quality of life, affordability, job market, education, healthcare, and diversity, among others.
* According to a report by U.S. News & World Report, the top 5 Best Cities in the US in 2024 are:
	+ Denver, CO: ranked #1 for its strong economy, high quality of life, and easy access to outdoor recreation.
	+ Austin, TX: ranked #2 for its vibrant music scene, diverse economy, and a highly educated population.
	+ Raleigh-Durham, NC: ranked #3 for its research triangle park, affordable housing, and a strong job market.
	+ Portland, OR: ranked #4 for its eco-friendly lifestyle, cultural attractions, and a thriving food scene.
	+ Seattle, WA: ranked #5 for its tech industry, outdoor activities, and a diverse population.
* Addi

As you can see from the results above, the AI Agents work together to create a report on the Top 5 cities in the US, all backed by a private large language model hosted with Cloudera's AI Inference Service. The agents used their trained knowledge to complete the task, but they can be configured with RAG tools to retrieve the latest information from the internet or other sources.